<a href="https://colab.research.google.com/github/asherzohara/Python/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training Demo
This is a simple example for training the SimSwap 224*224 with VGGFace2-224.

Code path: https://github.com/neuralchen/SimSwap
If you like the SimSwap project, please star it!
Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

In [1]:
!nvidia-smi

Mon Apr 28 09:14:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Installation
All file changes made by this notebook are temporary. You can try to mount your own google drive to store files if you want.

#Get Scripts

In [2]:
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

Cloning into 'SimSwap'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (501/501), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 1141 (delta 416), reused 383 (delta 383), pack-reused 640 (from 2)
Receiving objects: 100% (1141/1141), 211.46 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (602/602), done.
Already up to date.


# Install Blocks

In [3]:
!pip install googledrivedownloader
!pip install timm==0.5.4
!wget -P SimSwap/arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

#Download the Training Dataset
We employ the cropped VGGFace2-224 dataset for this toy training demo.

You can download the dataset from our google driver https://drive.google.com/file/d/19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc/view?usp=sharing

***Please check the dataset in dir /content/TrainingData***

***If dataset already exists in /content/TrainingData, please do not run blow scripts!***


In [6]:
%mkdir /content/TrainingData

In [7]:
!pip install gdown

In [8]:
!gdown --id 19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc -O /content/TrainingData/vggface2_crop_arcfacealign_224.tar

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc
From (redirected): https://drive.google.com/uc?id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc&confirm=t&uuid=0a25c45b-47c5-4de7-93d6-326856864b30
To: /content/TrainingData/vggface2_crop_arcfacealign_224.tar
100% 11.7G/11.7G [02:37<00:00, 74.4MB/s]


In [5]:
%mkdir /content/TrainingData
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19pWvdEHS-CEG6tW3PdxdtZ5QEymVjImc" -O /content/TrainingData/vggface2_crop_arcfacealign_224.tar && rm -rf /tmp/cookies.txt
%cd /content/
!tar -xzvf /content/TrainingData/vggface2_crop_arcfacealign_224.tar -C /content/TrainingData
 !rm /content/TrainingData/vggface2_crop_arcfacealign_224.tar

IndentationError: unexpected indent (<ipython-input-5-0c993758d7fe>, line 5)

#Trainig
Batch size must larger than 1!

In [9]:
%cd /content/SimSwap
!ls
!python train.py --name simswap224_test --gpu_ids 0 --dataset /content/TrainingData/vggface2_crop_arcfacealign_224 --Gdeep False

/content/SimSwap
 arcface_model	       README.md
 cog.yaml	      'SimSwap colab.ipynb'
 crop_224	       simswaplogo
 data		       test_one_image.py
 demo_file	       test_video_swapmulti.py
 docs		       test_video_swap_multispecific.py
 download-weights.sh   test_video_swapsingle.py
 insightface_func      test_video_swapspecific.py
 LICENSE	       test_wholeimage_swapmulti.py
 models		       test_wholeimage_swap_multispecific.py
 MultiSpecific.ipynb   test_wholeimage_swapsingle.py
 options	       test_wholeimage_swapspecific.py
 output		       train.ipynb
 parsing_model	       train.py
 pg_modules	       util
 predict.py
2025-04-28 09:20:11.616250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745832011.842919    2922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already b